Link 1 https://cantonga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_cantonga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[1]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[2]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[3]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(40)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://cantonga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Canton GA Open Records Request form...")
        result = handle_cantonga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing cantonga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[1]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Filled request details
Available options in dropdown: ['Only to review / inspect', 'Electronic copies when available; otherwise paper copies', 'Paper copies']
Selected dropdown option
Clicked submit button
Results saved to 

link2 https://maconbibbcountyga.justfoia.com/Forms/Launch/a709d888-de2f-4857-9c82-b12b2645a87c

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_maconbibbcountyga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div/div[1]/input": form_data["name"],       
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div/div[1]/input": form_data["zip"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div/div[1]/input": form_data["phone"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[13]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[14]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(40)

        # Extract confirmation message
        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://maconbibbcountyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Macon-Bibb County GA Open Records Request form...")
        result = handle_maconbibbcountyga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing maconbibbcountyga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Filled request details
Available options in dropdown: ['Only to review / inspect', 'Electronic copies when available; otherwise paper copies', 'Paper copies']
Selected dropdown optio

Link 3 https://woodstockga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_woodstockga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[1]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[3]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[2]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("View Only")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(40)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://woodstockga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Woodstock GA Open Records Request form...")
        result = handle_woodstockga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing woodstockga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[1]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Filled request details
Available options in dropdown: ['View Only', 'View/Copy', 'Digital if available', 'Copies Only']
Selected dropdown option
Clicked submit button
Results saved to submission_results.csv
Status: Succe

Link 4 https://smyrnaga.justfoia.com/Forms/Launch/fd208f47-7557-4edf-9478-723c87ba6b30

In [ ]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "first name": "Joe",
    "last name": "Doe",
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_smyrnaga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div/div[1]/input": form_data["first name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div/div[1]/input": form_data["last name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[15]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[14]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review/inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        
        # Delay of 15 seconds after form submission
        time.sleep(40)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://smyrnaga.justfoia.com/Forms/Launch/fd208f47-7557-4edf-9478-723c87ba6b30"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Smyrna GA Open Records Request form...")
        result = handle_smyrnaga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing Smyrnaga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div/div[1]/input
Filled request details
Available options in dropdown: ['Email (Electronic/Scanned C

Link 5 https://austellga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_austellga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(60)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://austellga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Austell GA Open Records Request form...")
        result = handle_austellga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing austellga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Filled request details
Available options in dropdown: ['Electronic copies when available; otherwise physical copies', 'Only to review / inspect', 'Physical copies for pickup', 'Physical copies

Link 6 https://albanyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_albanyga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[15]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Handle checkboxes
        checkbox1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[17]/div/div[1]/div/div/div/div/div/div/div"
        checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox1_xpath)))
        driver.execute_script("arguments[0].click();", checkbox)
        print(f"Checked {checkbox1_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(60)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://albanyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Albany GA Open Records Request form...")
        result = handle_albanyga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing albanyga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/input
Filled request details
Available options in dropdown: ['Only to review / inspect', 'US Mail', 'Electronic Copies']
Selected dropdown option
Checked /html/body/div[1]/div[2]/main/div/div[1]/for

Link 7 https://forsythcountyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_forsythcountyga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[13]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[14]/div/div[1]/div/div/div/div/div/div/div"
        checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox1_xpath)))
        driver.execute_script("arguments[0].click();", checkbox)
        print(f"Checked {checkbox1_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(20)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your request reference number is')]").text
            security_key_match = re.search(r"Your security key is (\S+)", confirmation_message)
            reference_number_match = re.search(r"Your request reference number is (\S+)", confirmation_message)
            if security_key_match:
                security_key = security_key_match.group(1)
            else:
                security_key = ""
                print("Security key not found in confirmation message")
            if reference_number_match:
                reference_number = reference_number_match.group(1)
            else:
                reference_number = ""
                print("Reference number not found in confirmation message")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://forsythcountyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Forsyth County GA Open Records Request form...")
        result = handle_forsythcountyga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing forsythcountyga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/input
Filled request details
Available options in dropdown: ['Only to review / inspect', 'Electronic copies when available; otherwise paper copies', 'Paper copies']
Selected dropdown option
C

Link 8 https://fairburnga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [11]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_fairburnga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div/div[1]/div/div/div/div/div/div/div"
        for checkbox_xpath in [checkbox1_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(30)

        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://fairburnga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing fairburnga Open Records Request form...")
        result = handle_fairburnga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing fairburnga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/input
Filled request details
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div[1]/div/div/div/div/div/div/div
Clicked submit button
Results saved to submission_results.csv
Status: Success
Form submit

Link 9 https://collegeparkga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_collegeparkga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[2]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[3]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}
    except Exception as e:
            print(f"Error in form handling: {str(e)}")
            driver.save_screenshot("error_main.png")
            return {"status": "Failed", "error": str(e)}
def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://collegeparkga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing College Park GA Open Records Request form...")
        result = handle_collegeparkga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing collegeparkga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Filled request details
Available options in dropdown: ['Only to review / inspect', 'Electronic copies when available; otherwise paper copies', 'Paper copies']
Selected dropdown option
Clicke

Link 10 https://conyersga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_conyersga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div[1]/div/div/div/div/div/div/div"
        for checkbox_xpath in [checkbox1_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(30)

        # Delay of 15 seconds after form submission
        time.sleep(20)

        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://conyersga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing conyersga Open Records Request form...")
        result = handle_conyersga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing conyersga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/input
Filled request details
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div[1]/div/div/div/div/div/div/div
Clicked submit button
Results saved to submission_results.csv
Status: Success
Form submitt

Link 11 https://spaldingcountyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_spaldingcountyga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[10]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[11]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[12]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(30)

        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://spaldingcountyga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing spaldingcountyga Open Records Request form...")
        result = handle_spaldingcountyga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing spaldingcountyga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Filled request details
Available options in dropdown: ['Paper copies', 'Certified copies', 'Only to review / inspect', 'Electronic copies when available; otherwise paper copies']
Select

Link 12 https://cityofaugustaga.nextrequest.com/requests/new

In [15]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "company":"no",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_cityofaugustaga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
        
            
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[1]/div/input": form_data["email"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[2]/div/input": form_data["name"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[3]/div/input": form_data["phone"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[4]/div/textarea": form_data["address"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[1]/div/div/input": form_data["city"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[3]/div/div/input": form_data["zip"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[6]/div/input": form_data["company"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[1]/div[2]/div[1]/div[2]/div/div[1]"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[2]/div/div/div/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Alabama")  # Adjust text as per available options
        print("Selected dropdown option")
#checkbox
        checkbox1_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[3]/div[2]/div/div/input"
        for checkbox_xpath in [checkbox1_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[4]/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://cityofaugustaga.nextrequest.com/requests/new"

    driver = None
    try:
        driver = setup_driver()
        print("Processing cityofaugustaga Open Records Request form...")
        result = handle_cityofaugustaga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Processing cityofaugustaga Open Records Request form...
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[1]/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[2]/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[3]/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[4]/div/textarea
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[1]/div/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[3]/div/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[6]/div/input
Filled request details
Available options in dropdown: ['-', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho

Link 13 https://cityoffayettevillega.nextrequest.com/requests/new

In [19]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "company": "no",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_cityoffayettevillega_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[1]/div/input": form_data["email"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[2]/div/input": form_data["name"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[3]/div/input": form_data["phone"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[4]/div/textarea": form_data["address"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[1]/div/div/input": form_data["city"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[3]/div/div/input": form_data["zip"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[6]/div/input": form_data["company"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[1]/div[2]/div[1]/div[2]/div/div[1]"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle department dropdown/autocomplete
        dropdown1_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[1]/div[4]/div/div/div/div/div[1]/input"
        dropdown1 = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown1_xpath)))
        
        # Clear existing value
        dropdown1.clear()
        
        # Click to focus
        dropdown1.click()
        time.sleep(1)
        
        # Type value
        dropdown1.send_keys("City Hall")
        time.sleep(1)
        
        # Press down arrow and enter to select from autocomplete
        actions = ActionChains(driver)
        actions.send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
        time.sleep(1)
        print("Entered 'City Hall' in department field")

        # Handle state dropdown
        dropdown2_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[2]/div/div/div/select"
        dropdown2 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown2_xpath))))
        dropdown2.select_by_visible_text("Alabama")
        print("Selected state dropdown option")

        # Handle checkbox
        checkbox1_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[3]/div[2]/div/div/input"
        checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox1_xpath)))
        driver.execute_script("arguments[0].click();", checkbox)
        print("Checked agreement checkbox")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[4]/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://cityoffayettevillega.nextrequest.com/requests/new"

    driver = None
    try:
        driver = setup_driver()
        print("Processing cityoffayettevillega Open Records Request form...")
        result = handle_cityoffayettevillega_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing cityoffayettevillega Open Records Request form...
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[1]/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[2]/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[3]/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[4]/div/textarea
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[1]/div/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[3]/div/div/input
Successfully filled field: /html/body/main/div/main/section/div/div[1]/form/div[2]/div[6]/div/input
Filled request details
Entered 'City Hall' in department field
Selected state dropdown option
Checked agreement checkbox
Clicked submit button
Results saved to submission_results.csv
Status: Success
Form submitted 

Link 14 https://unioncityga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [6]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_unioncityga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")
        
        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div[1]/div/div/div/div/div/div/div"
        checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox1_xpath)))
        driver.execute_script("arguments[0].click();", checkbox)
        print(f"Checked {checkbox1_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 30 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your request reference number is')]").text
            print(f"Confirmation message: {confirmation_message}")
            security_key_match = re.search(r"Your security key is (\S+)", confirmation_message)
            reference_number_match = re.search(r"Your request reference number is (\S+)", confirmation_message)
            security_key = security_key_match.group(1) if security_key_match else ""
            reference_number = reference_number_match.group(1) if reference_number_match else ""
            if not security_key or not reference_number:
                print("Confirmation message found but could not extract reference number or security key")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://unioncityga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Union City GA Open Records Request form...")
        result = handle_unioncityga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing Union City GA Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input
Filled request details
Available options in dropdown: ['Only to review / inspect', 'Electronic copies when available; otherwise paper copies', 'Paper copies']
Selected dropdown option
Checked /html/body/div[1]/div[2]/m

Link 15 https://hapevillega.justfoia.com/Forms/Launch/0e8fe236-181b-486e-8bb2-33990d9295c0

In [4]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_hapevillega_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[1]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")
        
        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/div/div/div/div/div/div/div"
        checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox1_xpath)))
        driver.execute_script("arguments[0].click();", checkbox)
        print(f"Checked {checkbox1_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your request reference number is')]").text
            print(f"Confirmation message: {confirmation_message}")
            security_key_match = re.search(r"Your security key is (\S+)", confirmation_message)
            reference_number_match = re.search(r"Your request reference number is (\S+)", confirmation_message)
            security_key = security_key_match.group(1) if security_key_match else ""
            reference_number = reference_number_match.group(1) if reference_number_match else ""
            if not security_key or not reference_number:
                print("Confirmation message found but could not extract reference number or security key")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://hapevillega.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Hapeville GA Open Records Request form...")
        result = handle_hapevillega_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing hapevillega Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input
Filled request details
Available options in dropdown: ['Insurance Company', 'Conducting Research', 'Media', 'Lawyer/ Legal Represenative', 'Witness To Accident', 'Involved Party', 'Law Enforcement/Other Government Agency

Link 16 https://fairburnga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [5]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_fairburnga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        #checkbox
        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div[1]/div/div/div/div/div/div/div"
        for checkbox_xpath in [checkbox1_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission3_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://fairburnga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing fairburnga Open Records Request form...")
        result = handle_fairburnga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Processing fairburnga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/input
Filled request details
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div[1]/div/div/div/div/div/div/div
Clicked submit button
Results saved to submission3_results.csv
Status: Success
Form submi

Link 17 https://eastpointga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [5]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
   "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_eastpointga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[13]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[15]/div/div[1]/div/div/div/div/div/div/div"
        for checkbox_xpath in [checkbox1_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(60)

        # Extract confirmation message
        confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your request reference number')]").text
        reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
        security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)

        return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://eastpointga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing eastpointga Open Records Request form...")
        result = handle_eastpointga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        # Do not close the driver
        pass

if __name__ == "__main__":
    main()

Processing eastpointga Open Records Request form...
Error filling field /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input: Message: 
Stacktrace:
	GetHandleVerifier [0x002EFD53+23747]
	(No symbol) [0x00277D54]
	(No symbol) [0x0014BE53]
	(No symbol) [0x0018FCA6]
	(No symbol) [0x0018FEEB]
	(No symbol) [0x001CD852]
	(No symbol) [0x001B1E44]
	(No symbol) [0x001CB41E]
	(No symbol) [0x001B1B96]
	(No symbol) [0x00183F3C]
	(No symbol) [0x00184EBD]
	GetHandleVerifier [0x005CAC73+3017699]
	GetHandleVerifier [0x005DB93B+3086507]
	GetHandleVerifier [0x005D40F2+3055714]
	GetHandleVerifier [0x00385AF0+637536]
	(No symbol) [0x00280A5D]
	(No symbol) [0x0027DA28]
	(No symbol) [0x0027DBC5]
	(No symbol) [0x002707F0]
	BaseThreadInitThunk [0x75A5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x773C809E+286]
	RtlGetAppContainerNamedObjectPath [0x773C806E+238]
	(No symbol) [0x00000000]

Error filling field /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/

Link 18 https://sandyspringsga.justfoia.com/Forms/Launch/9739a52c-6627-4522-acf8-5f87f95f4a9d

In [10]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "case": "no",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_sandyspringsga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input": form_data["zip"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[15]/div/div[1]/input": form_data["case"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[17]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("No")  # Adjust text as per available options
        print("Selected dropdown option")

        dropdown2_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[16]/div/div[1]/select"
        dropdown2 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown2_xpath))))
        available_options = [o.text.strip() for o in dropdown2.options]
        print("Available options in dropdown:", available_options)
        dropdown2.select_by_visible_text("N/A - I am not requesting an accident report")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 30 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Request number:')]").text
            print(f"Confirmation message: {confirmation_message}")
            security_key_match = re.search(r"Security key: (\S+)", confirmation_message)
            reference_number_match = re.search(r"Request number: (\S+)", confirmation_message)
            security_key = security_key_match.group(1) if security_key_match else ""
            reference_number = reference_number_match.group(1) if reference_number_match else ""
            if not security_key or not reference_number:
                print("Confirmation message found but could not extract reference number or security key")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://sandyspringsga.justfoia.com/Forms/Launch/9739a52c-6627-4522-acf8-5f87f95f4a9d"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Sandy Springs GA Open Records Request form...")
        result = handle_sandyspringsga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing Sandy Springs GA Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[15]/div/div[1]/input
Filled request details
Available options in dropdown: ['No', 'Yes']
Selected 

Link 19 https://roswellga.justfoia.com/Forms/Launch/02b6cc8e-5783-45b0-9fbb-3779b3387e72

In [12]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_roswellga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[1]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div[1]/div/div/div/div/div/div/div"
        checkbox2_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[19]/div[1]/div/div/div/div/div/div/div"
        checkbox3_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[21]/div[1]/div/div/div/div/div/div/div"
        checkbox4_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[22]/div[1]/div/div/div/div/div/div/div"
        for checkbox_xpath in [checkbox1_xpath, checkbox2_xpath, checkbox3_xpath, checkbox4_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 30 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Request Number:')]").text
            print(f"Confirmation message: {confirmation_message}")
            security_key_match = re.search(r"Security Key:\s+(\S+)", confirmation_message)
            reference_number_match = re.search(r"Request Number:\s+(\S+)", confirmation_message)
            security_key = security_key_match.group(1) if security_key_match else ""
            reference_number = reference_number_match.group(1) if reference_number_match else ""
            if not security_key or not reference_number:
                print("Confirmation message found but could not extract reference number or security key")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://roswellga.justfoia.com/Forms/Launch/02b6cc8e-5783-45b0-9fbb-3779b3387e72"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Roswell GA Open Records Request form...")
        result = handle_roswellga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing Roswell GA Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[1]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input
Filled request details
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div[1]/div/div/div/div/div/div/div
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[19]/div[1]/div/div/di

link 20 https://alpharettaga.justfoia.com/Forms/Launch/39f66254-4f9f-4794-955f-a15b1afebd0e

In [23]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_alpharettaga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[20]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[23]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[24]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[25]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[26]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle checkbox selection
        checkbox1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[19]/div[1]/div/div/div/div/div/div/div"
        checkbox2_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/div/div/div/div/div/div/div"
        checkbox3_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div[1]/div/div/div/div/div/div/div"
        for checkbox_xpath in [checkbox1_xpath, checkbox2_xpath, checkbox3_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 30 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            print(f"Confirmation message: {confirmation_message}")
            security_key_match = re.search(r"Your security key is (\S+)", confirmation_message)
            reference_number_match = re.search(r"Your request reference number is (\S+)", confirmation_message)
            security_key = security_key_match.group(1) if security_key_match else ""
            reference_number = reference_number_match.group(1) if reference_number_match else ""
            if not security_key or not reference_number:
                print("Confirmation message found but could not extract reference number or security key")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}


def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://alpharettaga.justfoia.com/Forms/Launch/39f66254-4f9f-4794-955f-a15b1afebd0e"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Alpharetta GA Open Records Request form...")
        result = handle_alpharettaga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing Alpharetta GA Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[20]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[23]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[24]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[25]/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[26]/div[1]/input
Filled request details
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[19]/div[1]/div/div/div/div/div/div/div
Checked /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div[1]/di

Link 21 https://norcrossga.justfoia.com/Forms/Launch/2da76d30-7849-4d56-b182-ef68865e40f6

In [13]:
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "date": "13/12/2024",
    "time": "7:25 pm",
    "person represented": "no",
    "case number": "12",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_norcrossga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Other")  # Adjust text as per available options
        print("Selected dropdown option")

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input": form_data["case number"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 30 seconds after form submission
        time.sleep(30)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            print(f"Confirmation message: {confirmation_message}")
            security_key_match = re.search(r"Your security key is (\S+)", confirmation_message)
            reference_number_match = re.search(r"Your request reference number is (\S+)", confirmation_message)
            security_key = security_key_match.group(1) if security_key_match else ""
            reference_number = reference_number_match.group(1) if reference_number_match else ""
            if not security_key or not reference_number:
                print("Confirmation message found but could not extract reference number or security key")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'reference_number', 'security_key', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://norcrossga.justfoia.com/Forms/Launch/2da76d30-7849-4d56-b182-ef68865e40f6"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Norcross GA Open Records Request form...")
        result = handle_norcrossga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing norcrossga Open Records Request form...
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/input
Successfully filled field: /html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/di

Link 22 https://www.cityofgriffin.com/services/open-records

In [18]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "first name": "John",
    "last name": "Doe",
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "unit number": "12",
    "country": "USA",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}


def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_cityofgriffin_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[1]/vi-form-field-edit/vi-field-fullname-edit/div/span[1]/input": form_data["first name"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[1]/vi-form-field-edit/vi-field-fullname-edit/div/span[2]/input": form_data["last name"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[4]/vi-form-field-edit/vi-field-email-edit/div/input": form_data["email"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[3]/vi-form-field-edit/vi-field-phone-edit/div/span/input": form_data["phone"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/div[2]/ol/li[2]/input": form_data["email"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[2]/vi-form-field-edit/vi-field-fulladdress-edit/div/span[1]/input": form_data["address"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[2]/vi-form-field-edit/vi-field-fulladdress-edit/div/span[2]/input": form_data["unit number"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[2]/vi-form-field-edit/vi-field-fulladdress-edit/div/span[3]/span[1]/input": form_data["city"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[2]/vi-form-field-edit/vi-field-fulladdress-edit/div/span[3]/span[2]/input": form_data["state"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[2]/vi-form-field-edit/vi-field-fulladdress-edit/div/span[4]/input": form_data["zip"],
            "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[2]/vi-form-field-edit/vi-field-fulladdress-edit/div/span[5]/input": form_data["country"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[5]/vi-form-field-edit/vi-field-paragraphtext-edit/div/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[6]/vi-form-field-edit/vi-field-singledropdown-edit/div/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Yes")  # Adjust text as per available options
        print("Selected dropdown option")
      

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/form-builder-submit-actions/div/a[2]"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://www.cityofgriffin.com/services/open-records"

    driver = None
    try:
        driver = setup_driver()
        print("Processing cityofgriffin Open Records Request form...")
        result = handle_cityofgriffin_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Processing cityofgriffin Open Records Request form...
Successfully filled field: /html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[1]/vi-form-field-edit/vi-field-fullname-edit/div/span[1]/input
Successfully filled field: /html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[1]/vi-form-field-edit/vi-field-fullname-edit/div/span[2]/input
Successfully filled field: /html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[4]/vi-form-field-edit/vi-field-email-edit/div/input
Successfully filled field: /html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/form-builder-submit-pagination/div/div[3]/vi-form-field-edit/vi-field-phone-edit/div/span/input
Successfully filled field: /html/body/div[2]/div/div[2]/div[3]/div[2]/section/div/form/div/div/div[2]/ol/li[2]/input
Successfully filled field: /html/body/div[2]/div/div[2]/di

Link 23 https://henrycounty-services.app.transform.civicplus.com/forms/34175

In [19]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "first name": "John",
    "last name": "Doe",
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "unit number": "12",
    "country": "USA",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}


def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_henrycounty_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[3]/div/div/div[2]/div/input": form_data["first name"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[4]/div/div/div[2]/div/input": form_data["last name"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[6]/div/div/div[2]/div/input": form_data["email"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[11]/div/div/div[2]/div/input": form_data["phone"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[7]/div/div/div[2]/div/input": form_data["address"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[8]/div/div/div[2]/div/input": form_data["city"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[9]/div/div/div[2]/div/input": form_data["state"],
            "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[10]/div/div/div[2]/div/input": form_data["zip"],
          
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[13]/div/div/div[2]/textarea[1]"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[4]/button[2]/span/span"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://henrycounty-services.app.transform.civicplus.com/forms/34175"

    driver = None
    try:
        driver = setup_driver()
        print("Processing henrycounty Open Records Request form...")
        result = handle_henrycounty_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Processing henrycounty Open Records Request form...
Successfully filled field: /html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[3]/div/div/div[2]/div/input
Successfully filled field: /html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[4]/div/div/div[2]/div/input
Successfully filled field: /html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[6]/div/div/div[2]/div/input
Successfully filled field: /html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[11]/div/div/div[2]/div/input
Successfully filled field: /html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[7]/div/div/div[2]/div/input
Successfully filled field: /html/body/div/ob-bottom-navigation/div/div[1]/section/div/div/form/div/div[3]/div/div/div/div[8]/div/div/div[2]/div/input
Successfully filled field: /html/body/div/ob-bo

Link 25 https://fayettecountyga.gov/administration/open-records-request

In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "first name": "John",
    "last name": "Doe",
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "unit number": "12",
    "country": "USA",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}


def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_fayettecountyga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[1]": form_data["first name"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[2]": form_data["last name"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[5]": form_data["email"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[4]": form_data["phone"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[6]": form_data["email"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[7]": form_data["address"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[8]": form_data["city"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[9]": form_data["state"],
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[10]": form_data["zip"],
          
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/label[12]/input"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://fayettecountyga.gov/administration/open-records-request"

    driver = None
    try:
        driver = setup_driver()
        print("Processing fayettecountyga Open Records Request form...")
        result = handle_fayettecountyga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Processing fayettecountyga Open Records Request form...
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[1]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[2]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[5]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[4]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[6]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[7]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[8]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[9]
Successfully filled field: /html/body/div[1]/div/div[1]/div/div[5]/div/div[1]/div[1]/form/input[10]
Filled request details
Clicked submit button
Results

Link 26 https://fs6.formsite.com/mAFRD/jiupubq3at/index.html

In [2]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "date": "10/12/2024",
    "first name": "John",
    "last name": "Doe",
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "unit number": "12",
    "country": "USA",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_formsite_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/form/div[2]/div[6]/input": form_data["date"],
            "/html/body/form/div[2]/div[9]/input": form_data["name"],
            "/html/body/form/div[2]/div[22]/input": form_data["phone"],
            "/html/body/form/div[2]/div[21]/input": form_data["email"],
            "/html/body/form/div[2]/div[14]/input": form_data["address"],
            "//html/body/form/div[2]/div[17]/input": form_data["city"],
            "/html/body/form/div[2]/div[19]/input": form_data["zip"],
            "/html/body/form/div[2]/div[30]/input": form_data["date"],
            "/html/body/form/div[2]/div[40]/input": form_data["address"],
            "/html/body/form/div[2]/div[42]/input": form_data["city"],
            "/html/body/form/div[2]/div[44]/input": form_data["zip"],
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/form/div[2]/div[26]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle first dropdown
        try:
            dropdown0_xpath = "/html/body/form/div[2]/div[24]/select"
            dropdown0 = wait.until(EC.presence_of_element_located((By.XPATH, dropdown0_xpath)))
            select0 = Select(dropdown0)
            
            # Wait for options to load
            time.sleep(2)
            
            # Get available options
            options = [o for o in select0.options if o.text.strip()]
            if options:
                select0.select_by_visible_text(options[-1].text)  # Select last non-empty option
                print(f"Selected {options[-1].text} from first dropdown")
            else:
                print("No valid options found in first dropdown")
                
        except Exception as e:
            print(f"Error handling first dropdown: {str(e)}")
            driver.save_screenshot("dropdown0_error.png")

        # Handle state dropdowns
        for dropdown_xpath in ["/html/body/form/div[2]/div[18]/select", 
                             "/html/body/form/div[2]/div[43]/select"]:
            try:
                dropdown = wait.until(EC.presence_of_element_located((By.XPATH, dropdown_xpath)))
                select = Select(dropdown)
                
                # Wait for options to load
                time.sleep(2)
                
                # Try multiple selection methods
                try:
                    select.select_by_visible_text("Alabama")
                except:
                    try:
                        select.select_by_value("AL")
                    except:
                        try:
                            # Get all non-empty options
                            options = [o for o in select.options if o.text.strip()]
                            if options:
                                select.select_by_visible_text(options[1].text)  # Select first non-empty option
                            else:
                                print(f"No valid options found in dropdown {dropdown_xpath}")
                        except:
                            print(f"Could not select state in dropdown {dropdown_xpath}")
                            
                print(f"Handled state selection for {dropdown_xpath}")
                
            except Exception as e:
                print(f"Error handling state dropdown {dropdown_xpath}: {str(e)}")
                driver.save_screenshot(f"dropdown_error_{dropdown_xpath.replace('/', '_')}.png")

        # Handle checkboxes
        checkbox_xpaths = [
            "/html/body/form/div[2]/div[7]/table/tbody/tr[2]/td/label",
            "/html/body/form/div[2]/div[31]/table/tbody/tr[1]/td/label",
            "/html/body/form/div[2]/div[33]/table/tbody/tr[1]/td/label",
            "/html/body/form/div[2]/div[35]/table/tbody/tr[1]/td/label"
        ]
        
        for checkbox_xpath in checkbox_xpaths:
            try:
                checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
                driver.execute_script("arguments[0].click();", checkbox)
                print(f"Checked {checkbox_xpath}")
                time.sleep(1)  # Add small delay between checkbox clicks
            except Exception as e:
                print(f"Error checking checkbox {checkbox_xpath}: {str(e)}")
                driver.save_screenshot(f"checkbox_error_{checkbox_xpath.replace('/', '_')}.png")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")
        
        # Submit form
        submit_xpath = "/html/body/form/div[3]/div/input"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        
        # Wait for submission to complete
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://fs6.formsite.com/mAFRD/jiupubq3at/index.html"

    driver = None
    try:
        driver = setup_driver()
        print("Processing formsite Open Records Request form...")
        result = handle_formsite_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Processing formsite Open Records Request form...
Successfully filled field: /html/body/form/div[2]/div[6]/input
Successfully filled field: /html/body/form/div[2]/div[9]/input
Successfully filled field: /html/body/form/div[2]/div[22]/input
Successfully filled field: /html/body/form/div[2]/div[21]/input
Successfully filled field: /html/body/form/div[2]/div[14]/input
Successfully filled field: //html/body/form/div[2]/div[17]/input
Successfully filled field: /html/body/form/div[2]/div[19]/input
Successfully filled field: /html/body/form/div[2]/div[30]/input
Successfully filled field: /html/body/form/div[2]/div[40]/input
Successfully filled field: /html/body/form/div[2]/div[42]/input
Successfully filled field: /html/body/form/div[2]/div[44]/input
Filled request details
No valid options found in first dropdown
Handled state selection for /html/body/form/div[2]/div[18]/select
Handled state selection for /html/body/form/div[2]/div[43]/select
Checked /html/body/form/div[2]/div[7]/table/tbody/tr

Link 27 https://claytoncountywaterauthority.wufoo.com/forms/z1e6l46517vub7j/


In [3]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# Form data to submit
form_data = {
    "first name": "John",
    "last name": "Doe",
    "name": "John Doe",
    "phone": "123-456-7890",
    "p1": "123",
    "p2":"456",
    "p3":"7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "unit number": "12",
    "country": "USA",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}


def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_claytoncountywaterauthority_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/form/ul/li[1]/span[1]/input": form_data["first name"],
            "/html/body/div[1]/form/ul/li[1]/span[2]/input": form_data["last name"],
            "/html/body/div[1]/form/ul/li[5]/span[1]/input": form_data["p1"],
            "/html/body/div[1]/form/ul/li[5]/span[3]/input": form_data["p2"],
            "/html/body/div[1]/form/ul/li[5]/span[5]/input": form_data["p3"],
            "/html/body/div[1]/form/ul/li[6]/div/input": form_data["email"],
            "/html/body/div[1]/form/ul/li[4]/div/input": form_data["address"],
          
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/form/ul/li[7]/div/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/form/ul/li[8]/div/input[2]"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://claytoncountywaterauthority.wufoo.com/forms/z1e6l46517vub7j/"

    driver = None
    try:
        driver = setup_driver()
        print("Processing claytoncountywaterauthority Open Records Request form...")
        result = handle_claytoncountywaterauthority_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Processing claytoncountywaterauthority Open Records Request form...
Successfully filled field: /html/body/div[1]/form/ul/li[1]/span[1]/input
Successfully filled field: /html/body/div[1]/form/ul/li[1]/span[2]/input
Successfully filled field: /html/body/div[1]/form/ul/li[5]/span[1]/input
Successfully filled field: /html/body/div[1]/form/ul/li[5]/span[3]/input
Successfully filled field: /html/body/div[1]/form/ul/li[5]/span[5]/input
Successfully filled field: /html/body/div[1]/form/ul/li[6]/div/input
Successfully filled field: /html/body/div[1]/form/ul/li[4]/div/input
Filled request details
Clicked submit button
Results saved to submission2_results.csv
Status: Success
Form submitted successfully!


Link 28 https://peachtreecitygapolice.nextrequest.com/requests/new

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "company":"no",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_peachtreecitygapolice_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
        
            
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[1]/div/input": form_data["email"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[2]/div/input": form_data["name"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[3]/div/input": form_data["phone"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[4]/div/textarea": form_data["address"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[1]/div/div/input": form_data["city"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[3]/div/div/input": form_data["zip"],
            "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[6]/div/input": form_data["company"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[1]/div[2]/div[1]/div[2]/div/div[1]"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[2]/div[5]/div[2]/div/div/div/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Alabama")  # Adjust text as per available options
        print("Selected dropdown option")

                    # Handle dropdown selection (corrected)
        dropdown2_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[1]/div[4]/div/div/div/div/div[1]/input"
        dropdown2 = wait.until(EC.presence_of_element_located((By.XPATH, dropdown2_xpath)))



   # Clear the dropdown input field
        dropdown2.clear()

# Enter the desired option into the input field
        dropdown2.send_keys("Library")
        print("Typed 'Library' into the dropdown")

# Press Enter to confirm the selection
        dropdown2.send_keys(Keys.ENTER)
        print("Pressed Enter to select 'Library'")


        
        # check
        checkbox1_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[3]/div[2]/div/div/input"
        for checkbox_xpath in [checkbox1_xpath]:
            checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, checkbox_xpath)))
            driver.execute_script("arguments[0].click();", checkbox)
            print(f"Checked {checkbox_xpath}")
        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/main/div/main/section/div/div[1]/form/div[4]/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")
        time.sleep(5)
        return {"status": "Success", "confirmation": "Form submitted"}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://peachtreecitygapolice.nextrequest.com/requests/new"

    driver = None
    try:
        driver = setup_driver()
        print("Processing peachtreecitygapolice Open Records Request form...")
        result = handle_peachtreecitygapolice_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Link 29 https://riverdalega.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_riverdale_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[1]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[2]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[3]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(60)

        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://riverdalega.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Riverdale GA Open Records Request form...")
        result = handle_riverdale_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

link 30 https://forestparkga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_forestparkga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[4]/div/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[5]/div/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[6]/div/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[7]/div/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[8]/div/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[9]/div/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[10]/div/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[11]/div/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[2]/div/div[12]/div/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/main/div/div[1]/form/div[4]/div/button"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(15)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission2_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://forestparkga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing forestparkga Open Records Request form...")
        result = handle_forestparkga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

link 31 ackworthga

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
import re

# Form data to submit
form_data = {
    "name": "John Doe",
    "phone": "123-456-7890",
    "email": "dummyemail@example.com",
    "address": "123 Main Street, Suite 100",
    "city": "SampleCity",
    "state": "SampleState",
    "zip": "12345",
    "message": """My name is John Doe, and I am requesting records from the Code Enforcement Office.

In accordance with the [Insert Local Open Records Act], I am requesting a list of all open code violations for residential properties over the past 30 days. Specifically, I am interested in violations related to damaged or decayed roofs, mold, broken windows, boarded-up windows and doors, overgrown weeds and grass, trash and debris, rodent infestations or unsanitary conditions, flaking or peeling paint, vacant and unsecured structures, and any buildings deemed dangerous, uninhabitable, or unfit for occupancy.

Please provide the details of these violations, including the nature of the violation, the address of the property, and the date of the violation. If possible, I would appreciate receiving the information in a digital format, such as a .csv file or a searchable PDF. However, I am happy to accept the information in any format that is convenient for your office.

Thank you for your assistance in this matter.

Sincerely,
John Doe"""
}

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def fill_field(driver, wait, xpath, value):
    try:
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.clear()
        element.send_keys(value)
        print(f"Successfully filled field: {xpath}")
    except Exception as e:
        print(f"Error filling field {xpath}: {str(e)}")
        driver.save_screenshot(f"error_{xpath.replace('/', '_')}.png")

def handle_acworthga_form(driver, url):
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Wait for page to load completely
        time.sleep(7)

        # Fill all fields
        fields_to_fill = {
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[1]/div[1]/input": form_data["name"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[2]/div[1]/input": form_data["phone"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[3]/div[1]/input": form_data["email"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[4]/div[1]/input": form_data["address"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[5]/div[1]/input": form_data["city"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[6]/div[1]/input": form_data["state"],
            "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[7]/div[1]/input": form_data["zip"]
        }

        for xpath, value in fields_to_fill.items():
            fill_field(driver, wait, xpath, value)

        # Fill request details
        details_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[8]/div[1]/textarea"
        details = wait.until(EC.presence_of_element_located((By.XPATH, details_xpath)))
        details.clear()
        details.send_keys(form_data["message"])
        print("Filled request details")

        # Handle dropdown selection
        dropdown1_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[2]/div/div[9]/div[1]/select"
        dropdown1 = Select(wait.until(EC.presence_of_element_located((By.XPATH, dropdown1_xpath))))
        available_options = [o.text.strip() for o in dropdown1.options]
        print("Available options in dropdown:", available_options)
        dropdown1.select_by_visible_text("Only to review / inspect")  # Adjust text as per available options
        print("Selected dropdown option")

        # Take screenshot before submission
        driver.save_screenshot("before_submit.png")

        # Submit form
        submit_xpath = "/html/body/div[1]/div[2]/div[1]/form/main/div/div[4]/div/button/div"
        submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, submit_xpath)))
        driver.execute_script("arguments[0].click();", submit_button)
        print("Clicked submit button")

        # Delay of 15 seconds after form submission
        time.sleep(60)

        # Extract confirmation message
        try:
            confirmation_message = driver.find_element(By.XPATH, "//*[contains(text(), 'Your security key is')]").text
            security_key = re.search(r"Your security key is (\S+)", confirmation_message).group(1)
            reference_number = re.search(r"Your request reference number is (\S+)", confirmation_message).group(1)
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": reference_number, "security_key": security_key}
        except NoSuchElementException:
            print("Confirmation message not found")
            return {"status": "Success", "confirmation": "Form submitted", "reference_number": "", "security_key": ""}

    except Exception as e:
        print(f"Error in form handling: {str(e)}")
        driver.save_screenshot("error_main.png")
        return {"status": "Failed", "error": str(e)}

def save_results(results, filename="submission_results.csv"):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['url', 'status', 'confirmation', 'error', 'reference_number', 'security_key'])
            writer.writeheader()
            writer.writerows(results)
        print(f"Results saved to {filename}")
    except Exception as e:
        print(f"Error saving results: {e}")

def main():
    url = "https://acworthga.justfoia.com/Forms/Launch/d705cbd6-1396-49b7-939e-8d86c5a87deb"

    driver = None
    try:
        driver = setup_driver()
        print("Processing Acworth GA Open Records Request form...")
        result = handle_acworthga_form(driver, url)

        save_results([{
            "url": url,
            "status": result.get("status"),
            "confirmation": result.get("confirmation", ""),
            "error": result.get("error", ""),
            "reference_number": result.get("reference_number", ""),
            "security_key": result.get("security_key", "")
        }])

        print(f"Status: {result.get('status')}")
        if result.get('status') == 'Success':
            print("Form submitted successfully!")
        else:
            print(f"Error: {result.get('error')}")

    except WebDriverException as e:
        print(f"WebDriver error: {e}")

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()